<a href="https://colab.research.google.com/github/FunNyLuAz/Scripts-Universidade/blob/main/Python/Performance%20em%20Sistemas%20Ciberf%C3%ADsicos/07/Simulador%20Mapeamento%20Direto%20(modificado)/Simulador%20Mapeamento%20Direto%20(modificado).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

# Classe Cache para simular o funcionamento de uma cache de memória
class Cache:
    # Método construtor da classe Cache
    def __init__(self, tamanho_cache, tamanho_bloco, algoritmo_substituicao):
        self.tamanho_cache = tamanho_cache  # Número de blocos na cache
        self.tamanho_bloco = tamanho_bloco  # Tamanho de cada bloco na cache
        self.algoritmo_substituicao = algoritmo_substituicao  # Algoritmo de substituição a ser usado (LRU, LFU ou FIFO)
        # Inicializa a cache com blocos vazios
        self.cache = [{'validade': 0, 'tag': None, 'contador': 0, 'timestamp': 0, 'dados': [None] * tamanho_bloco} for _ in range(tamanho_cache)]
        self.acessos = 0  # Contador de acessos à cache

    # Método para decompor o endereço em tag, índice e deslocamento
    def decomporEndereço(self, endereco_binario):
        endereco = int(endereco_binario, 2)  # Converte o endereço binário para um inteiro
        indice = (endereco // self.tamanho_bloco) % self.tamanho_cache  # Calcula o índice do bloco na cache
        tag = endereco // (self.tamanho_bloco * self.tamanho_cache)  # Calcula a tag do endereço
        offset = endereco % self.tamanho_bloco  # Calcula o deslocamento dentro do bloco
        return tag, indice, offset  # Retorna a tag, índice e deslocamento

    # Método para buscar um dado na cache e retornar o dado e o status de cache hit ou miss
    def buscaDado(self, MemoriaPrincipal, endereco_binario):
        tag, indice, offset = self.decomporEndereço(endereco_binario)  # Decompõe o endereço
        entraCache = self.cache[indice]  # Seleciona a entrada da cache correspondente ao índice
        self.acessos += 1  # Incrementa o contador de acessos

        # Verifica se a entrada da cache é válida e se a tag corresponde
        if entraCache['validade'] and entraCache['tag'] == tag:
            entraCache['contador'] += 1  # Incrementa o contador de uso da entrada da cache
            entraCache['timestamp'] = self.acessos  # Atualiza o timestamp de acesso da entrada da cache
            return entraCache['dados'][offset], True  # Retorna o dado e o status de cache hit
        else:
            # Atualiza a entrada da cache com os novos dados
            entraCache['tag'] = tag
            entraCache['validade'] = 1
            entraCache['contador'] = 1
            entraCache['timestamp'] = self.acessos
            entraCache['dados'] = MemoriaPrincipal[endereco_binario][:self.tamanho_bloco]
            return entraCache['dados'][offset], False  # Retorna o dado e o status de cache miss

    # Método para encontrar o índice do bloco menos recentemente usado (LRU)
    def encontrarLRU(self):
        indice_minimo = min(self.cache, key=lambda x: x['timestamp'])  # Encontra o bloco com o menor timestamp
        return self.cache.index(indice_minimo)  # Retorna o índice do bloco LRU

    # Método para encontrar o índice do bloco menos frequentemente usado (LFU)
    def encontrarLFU(self):
        # Encontra o bloco com o menor contador e, em caso de empate, o menor timestamp
        indice_minimo = min(self.cache, key=lambda x: (x['contador'], x['timestamp']))
        return self.cache.index(indice_minimo)  # Retorna o índice do bloco LFU

    # Método para encontrar o índice do bloco que chegou primeiro (FIFO)
    def encontrarFIFO(self):
        indice_minimo = min(self.cache, key=lambda x: (x['timestamp'])) # Encontra o bloco com o menor timestamp
        return self.cache.index(indice_minimo)  # Retorna o índice do bloco FIFO

    # Método para encontrar o índice de um bloco aleatório (RAND)
    def encontrarRAND(self):
        indice_minimo = random.randint(0, self.tamanho_cache - 1) # Encontra um bloco aleatório
        return indice_minimo  # Retorna o índice do bloco RAND

    # Método para acessar um dado na memória e retornar o dado e o status de cache hit
    def acessar(self, MemoriaPrincipal, endereco_binario):
        dado, hit = self.buscaDado(MemoriaPrincipal, endereco_binario)  # Busca o dado na cache
        if not hit:  # Se ocorreu um cache miss
            self.substituirDado(MemoriaPrincipal, endereco_binario)  # Substitui o dado na cache
        return dado, hit  # Retorna o dado e o status de cache hit

    # Método para substituir um dado na cache usando o algoritmo de substituição especificado
    def substituirDado(self, MemoriaPrincipal, endereco_binario):
        # Escolhe o índice de substituição com base no algoritmo de substituição
        if self.algoritmo_substituicao == 'LRU':
            indice_substituicao = self.encontrarLRU()
        elif self.algoritmo_substituicao == 'LFU':
            indice_substituicao = self.encontrarLFU()
        elif self.algoritmo_substituicao == 'FIFO':
            indice_substituicao = self.encontrarFIFO()
        elif self.algoritmo_substituicao == 'RAND':
            indice_substituicao = self.encontrarRAND()
        else:
            raise ValueError("Algoritmo de substituição não suportado")  # Algoritmo não suportado

        tag, _, _ = self.decomporEndereço(endereco_binario)  # Decompõe o endereço para obter a tag
        # Atualiza a entrada da cache no índice de substituição com os novos dados
        self.cache[indice_substituicao]['tag'] = tag
        self.cache[indice_substituicao]['validade'] = 1
        self.cache[indice_substituicao]['contador'] = 1
        self.cache[indice_substituicao]['timestamp'] = self.acessos
        self.cache[indice_substituicao]['dados'] = MemoriaPrincipal[endereco_binario][:self.tamanho_bloco]


In [ ]:
import random

# Função para gerar uma memória principal fictícia
def criarMemoriaPrincipal(tamanhoMemoria, tamanho_dado):
    MemoriaPrincipal = {}
    for i in range(tamanhoMemoria):
        endereco_binario = format(i, 'b').zfill(16)  # Endereços binários de 16 bits
        # Gera dados aleatórios para cada endereço na memória principal
        MemoriaPrincipal[endereco_binario] = [random.randint(0, 255) for _ in range(tamanho_dado)]
    return MemoriaPrincipal

# Função para gerar uma sequência de acessos à memória
def gerarAcessosMemoria(num_acessos, tamanhoMemoria, padrao='aleatorio'):
    if padrao == 'aleatorio':
        # Gera uma lista de endereços de memória acessados aleatoriamente
        return [format(random.randint(0, tamanhoMemoria - 1), 'b').zfill(16) for _ in range(num_acessos)]
    elif padrao == 'localidade':
        acessos = []
        for _ in range(num_acessos // 10):  ##(throwaway variable)
            endereco_base = random.randint(0, tamanhoMemoria - 1)
            for i in range(10):
                endereco = (endereco_base + i) % tamanhoMemoria
                acessos.append(format(endereco, 'b').zfill(16))
        return acessos
    elif padrao == 'sequencial':
        # Gera uma lista de endereços de memória acessados sequencialmente
        return [format((i % tamanhoMemoria), 'b').zfill(16) for i in range(num_acessos)]
    else:
        raise ValueError("Padrão de acesso desconhecido")

# Parâmetros
tamanho_cache = 64
tamanho_bloco = 16
tamanhoMemoria = 4096
num_acessos = 1000

# Gerar memória principal
MemoriaPrincipal = criarMemoriaPrincipal(tamanhoMemoria, tamanho_bloco)

# Testar LRU e LFU para diferentes padrões de acesso
padroesAcesso = ['aleatorio', 'localidade', 'sequencial']
for padrao in padroesAcesso:
    print(f"\nPadrão de acesso: {padrao}")
    acessosMemoria = gerarAcessosMemoria(num_acessos, tamanhoMemoria, padrao)

    for algoritmo in ['LRU', 'LFU', 'FIFO', 'RAND']:
        # Inicializa a cache com o algoritmo de substituição correspondente
        cache = Cache(tamanho_cache, tamanho_bloco, algoritmo)
        hits = 0
        misses = 0

        # Acessa a memória usando a sequência de endereços gerada
        for endereco in acessosMemoria:
            _, hit = cache.acessar(MemoriaPrincipal, endereco)  ##(throwaway variable)
            if hit:
                hits += 1  # Incrementa o contador de cache hits
            else:
                misses += 1  # Incrementa o contador de cache misses

        # Imprime os resultados para o algoritmo e padrão de acesso atuais
        print(f"Algoritmo {algoritmo}: Hits = {hits}, Misses = {misses}, Taxa de acerto = {hits / num_acessos:.2%}")



Padrão de acesso: aleatorio
Algoritmo LRU: Hits = 225, Misses = 775, Taxa de acerto = 22.50%
Algoritmo LFU: Hits = 229, Misses = 771, Taxa de acerto = 22.90%
Algoritmo FIFO: Hits = 225, Misses = 775, Taxa de acerto = 22.50%
Algoritmo RAND: Hits = 234, Misses = 766, Taxa de acerto = 23.40%

Padrão de acesso: localidade
Algoritmo LRU: Hits = 876, Misses = 124, Taxa de acerto = 87.60%
Algoritmo LFU: Hits = 879, Misses = 121, Taxa de acerto = 87.90%
Algoritmo FIFO: Hits = 876, Misses = 124, Taxa de acerto = 87.60%
Algoritmo RAND: Hits = 870, Misses = 130, Taxa de acerto = 87.00%

Padrão de acesso: sequencial
Algoritmo LRU: Hits = 968, Misses = 32, Taxa de acerto = 96.80%
Algoritmo LFU: Hits = 968, Misses = 32, Taxa de acerto = 96.80%
Algoritmo FIFO: Hits = 968, Misses = 32, Taxa de acerto = 96.80%
Algoritmo RAND: Hits = 958, Misses = 42, Taxa de acerto = 95.80%
